## 1. Introduction to ReAct <a id="introduction"></a>

### What is ReAct?

**ReAct** (Reasoning and Acting) is a framework that combines:
- **Reasoning**: Generating thought traces that explain decision-making
- **Acting**: Taking actions in the environment using available tools

The key innovation is the interleaving of reasoning and action, creating a loop:
```
Thought → Action → Observation → Thought → Action → ...
```

### ReAct vs. Traditional Approaches

| Approach | Reasoning | Actions | Interleaved |
|----------|-----------|---------|-------------|
| Chain-of-Thought | ✓ | ✗ | ✗ |
| Action-Only Agents | ✗ | ✓ | ✗ |
| **ReAct** | **✓** | **✓** | **✓** |

### Benefits of ReAct

1. **Interpretability**: Reasoning traces explain why actions are taken
2. **Diagnosability**: Easy to identify where reasoning or execution fails
3. **Controllability**: Can guide behavior by modifying reasoning prompts
4. **Flexibility**: Adapts to new tools and environments easily
5. **Human-like**: Mirrors how humans think and act on problems

### Core Components

1. **Thought**: Internal reasoning about what to do next
2. **Action**: Specific tool or function call with parameters
3. **Observation**: Result returned from the action
4. **Decision**: Whether to continue or conclude

### Example ReAct Trace

```
Question: What is the capital of the country where the 2024 Olympics were held?

Thought: I need to first find out which country hosted the 2024 Olympics.
Action: search("2024 Olympics host country")
Observation: The 2024 Summer Olympics were held in Paris, France.

Thought: Now I know France hosted the Olympics. The capital of France is Paris.
Action: finish("Paris")
```

## 2. Environment Setup <a id="setup"></a>

Let's set up our environment and create some simple tools for the agent to use.

In [ ]:
#r "nuget: Azure.AI.Projects, 1.0.0-beta.4"
#r "nuget: Azure.Identity, 1.13.1"
#r "nuget: DotNetEnv, 3.1.1"
#r "nuget: Azure.AI.Inference, 1.0.0-beta.2"

using Azure.AI.Projects;
using Azure.Identity;
using DotNetEnv;
using System.Text.Json;
using System.Text.RegularExpressions;
using Azure.AI.Inference;

// Load environment variables
Env.Load();

// Initialize the AI Project client
var projectEndpoint = Environment.GetEnvironmentVariable("PROJECT_ENDPOINT");
var projectClient = new AIProjectClient(new Uri(projectEndpoint), new DefaultAzureCredential());

// Get the OpenAI client
var chat = projectClient.GetChatClient();
var model = Environment.GetEnvironmentVariable("MODEL");

Console.WriteLine("Connected to Azure AI Foundry");
Console.WriteLine($"Using model: {model}");

In [ ]:
// Knowledge base entry class
public class KnowledgeEntry
{
    public string Description { get; set; }
    public List<string> Features { get; set; }
    public List<string> Benefits { get; set; }
    public List<string> Components { get; set; }
    public List<string> Capabilities { get; set; }
    public int ReleaseYear { get; set; }
    public int PaperYear { get; set; }
    public string KeyInnovation { get; set; }
    public string InventedBy { get; set; }
}

// Create a simple knowledge base for our tools to access
var knowledgeBase = new Dictionary<string, KnowledgeEntry>
{
    ["azure_ai_foundry"] = new KnowledgeEntry
    {
        Description = "Azure AI Foundry is a platform for building, deploying, and managing AI applications",
        Features = new List<string> { "model deployment", "prompt engineering", "evaluation", "monitoring" },
        ReleaseYear = 2023
    },
    ["rag"] = new KnowledgeEntry
    {
        Description = "Retrieval-Augmented Generation combines retrieval with text generation",
        Benefits = new List<string> { "grounded responses", "reduced hallucination", "up-to-date information" },
        Components = new List<string> { "retriever", "generator", "knowledge base" }
    },
    ["react_framework"] = new KnowledgeEntry
    {
        Description = "ReAct combines reasoning and acting in an interleaved manner",
        PaperYear = 2022,
        KeyInnovation = "synergistic combination of reasoning traces and actions"
    },
    ["chain_of_thought"] = new KnowledgeEntry
    {
        Description = "Chain-of-Thought prompting encourages step-by-step reasoning",
        InventedBy = "Google Research",
        PaperYear = 2022
    },
    ["microsoft_agent_framework"] = new KnowledgeEntry
    {
        Description = "Framework for building autonomous AI agents",
        Features = new List<string> { "tool usage", "planning", "memory", "multi-agent orchestration" }
    },
    ["gpt4"] = new KnowledgeEntry
    {
        Description = "GPT-4 is a large multimodal model from OpenAI",
        ReleaseYear = 2023,
        Capabilities = new List<string> { "text generation", "image understanding", "code generation" }
    }
};

Console.WriteLine($"Knowledge base loaded with {knowledgeBase.Count} entries");

In [ ]:
// Tool function delegates
public delegate string ToolFunction(params string[] args);

// Tool registry entry
public class ToolInfo
{
    public ToolFunction Function { get; set; }
    public string Description { get; set; }
}

// Define simple tools for the agent
string SearchKnowledge(params string[] args)
{
    var query = args[0].ToLower();
    
    foreach (var kvp in knowledgeBase)
    {
        var key = kvp.Key.Replace("_", " ");
        if (key.Contains(query) || query.Contains(key) || kvp.Key.Split('_').Any(word => query.Contains(word)))
        {
            var options = new JsonSerializerOptions { WriteIndented = true };
            return JsonSerializer.Serialize(kvp.Value, options);
        }
    }
    
    var topics = string.Join(", ", knowledgeBase.Keys.Select(k => k.Replace("_", " ")));
    return $"Not found. Available topics: {topics}";
}

string Calculate(params string[] args)
{
    var expression = args[0];
    
    try
    {
        var allowedChars = "0123456789+-*/()., ";
        if (!expression.All(c => allowedChars.Contains(c)))
        {
            return "Error: Only basic arithmetic operations allowed (+, -, *, /, ())";
        }
        
        var dataTable = new System.Data.DataTable();
        var result = dataTable.Compute(expression, "");
        return result.ToString();
    }
    catch (Exception ex)
    {
        return $"Error: {ex.Message}";
    }
}

string GetYearDifference(params string[] args)
{
    try
    {
        var year1 = int.Parse(args[0]);
        var year2 = int.Parse(args[1]);
        var diff = Math.Abs(year1 - year2);
        return $"{diff} years";
    }
    catch (Exception ex)
    {
        return $"Error: {ex.Message}";
    }
}

string Finish(params string[] args)
{
    return $"FINAL_ANSWER: {args[0]}";
}

// Tool registry
var TOOLS = new Dictionary<string, ToolInfo>
{
    ["search_knowledge"] = new ToolInfo
    {
        Function = SearchKnowledge,
        Description = "Search the knowledge base for information about AI topics. Input: query string."
    },
    ["calculate"] = new ToolInfo
    {
        Function = Calculate,
        Description = "Perform mathematical calculations. Input: arithmetic expression as string."
    },
    ["get_year_difference"] = new ToolInfo
    {
        Function = GetYearDifference,
        Description = "Calculate difference between two years. Input: year1, year2."
    },
    ["finish"] = new ToolInfo
    {
        Function = Finish,
        Description = "Provide the final answer when you have enough information. Input: answer string."
    }
};

string GetToolDescriptions()
{
    return string.Join("\n", TOOLS.Select(kvp => $"{kvp.Key}: {kvp.Value.Description}"));
}

Console.WriteLine("Tools available:");
Console.WriteLine(GetToolDescriptions());

## 3. Basic ReAct Pattern <a id="basic"></a>

Let's implement the basic ReAct loop: Thought → Action → Observation → repeat.

### 3.1 Simple ReAct Agent

In [ ]:
// Action result class
public class ActionInfo
{
    public string ToolName { get; set; }
    public string Argument { get; set; }
}

// Trace step class
public class TraceStep
{
    public int Iteration { get; set; }
    public string Response { get; set; }
    public ActionInfo Action { get; set; }
    public string Observation { get; set; }
    public bool IsError { get; set; }
}

// Agent result class
public class AgentResult
{
    public string Answer { get; set; }
    public int Iterations { get; set; }
    public List<TraceStep> Trace { get; set; }
    public string Status { get; set; }
    public int ErrorsEncountered { get; set; }
    public int ActionsTaken { get; set; }
    public int Reflections { get; set; }
}

ActionInfo ParseAction(string text)
{
    var actionPattern = @"Action:\s*(\w+)\s*\(([^)]+)\)";
    var match = Regex.Match(text, actionPattern, RegexOptions.IgnoreCase);
    
    if (match.Success)
    {
        return new ActionInfo
        {
            ToolName = match.Groups[1].Value,
            Argument = match.Groups[2].Value.Trim('"', '\'', ' ')
        };
    }
    
    return null;
}

string ExecuteAction(string toolName, string argument)
{
    if (!TOOLS.ContainsKey(toolName))
    {
        return $"Error: Unknown tool '{toolName}'. Available tools: {string.Join(", ", TOOLS.Keys)}";
    }
    
    try
    {
        if (toolName == "get_year_difference")
        {
            var args = argument.Split(',').Select(a => a.Trim()).ToArray();
            return TOOLS[toolName].Function(args);
        }
        else
        {
            return TOOLS[toolName].Function(argument);
        }
    }
    catch (Exception ex)
    {
        return $"Error executing {toolName}: {ex.Message}";
    }
}

async Task<AgentResult> SimpleReactAgent(string question, int maxIterations = 5, bool verbose = true)
{
    var reactPrompt = $@"Answer the following question using this format:

Question: [the question]
Thought: [your reasoning about what to do]
Action: tool_name(""argument"")
Observation: [result from the action]
... (repeat Thought/Action/Observation as needed)
Thought: [final reasoning]
Action: finish(""final answer"")

Available tools:
{GetToolDescriptions()}

Question: {question}
";
    
    var conversation = reactPrompt;
    var trace = new List<TraceStep>();
    
    if (verbose)
    {
        Console.WriteLine(new string('=', 80));
        Console.WriteLine($"Question: {question}");
        Console.WriteLine(new string('=', 80));
        Console.WriteLine();
    }
    
    for (int iteration = 0; iteration < maxIterations; iteration++)
    {
        var requestOptions = new ChatCompletionsOptions
        {
            Messages = { new ChatRequestUserMessage(conversation) },
            Temperature = 0.3f,
            MaxTokens = 300
        };
        
        var response = await chat.CompleteChatAsync(model, requestOptions);
        var responseText = response.Value.Choices[0].Message.Content;
        
        if (verbose)
        {
            Console.WriteLine(responseText);
            Console.WriteLine();
        }
        
        var step = new TraceStep { Iteration = iteration + 1, Response = responseText };
        trace.Add(step);
        
        if (responseText.Contains("FINAL_ANSWER:"))
        {
            var finalAnswer = responseText.Split("FINAL_ANSWER:")[1].Trim();
            return new AgentResult
            {
                Answer = finalAnswer,
                Iterations = iteration + 1,
                Trace = trace,
                Status = "success"
            };
        }
        
        var actionInfo = ParseAction(responseText);
        
        if (actionInfo != null)
        {
            var observation = ExecuteAction(actionInfo.ToolName, actionInfo.Argument);
            
            if (verbose)
            {
                Console.WriteLine($"Observation: {observation}");
                Console.WriteLine(new string('-', 80));
                Console.WriteLine();
            }
            
            conversation += $"\n{responseText}\nObservation: {observation}\n";
            step.Action = actionInfo;
            step.Observation = observation;
            
            if (actionInfo.ToolName == "finish")
            {
                return new AgentResult
                {
                    Answer = observation.Replace("FINAL_ANSWER: ", ""),
                    Iterations = iteration + 1,
                    Trace = trace,
                    Status = "success"
                };
            }
        }
        else
        {
            if (verbose)
            {
                Console.WriteLine("Warning: No valid action found in response");
            }
            conversation += $"\n{responseText}\n";
        }
    }
    
    return new AgentResult
    {
        Answer = "Max iterations reached without finding answer",
        Iterations = maxIterations,
        Trace = trace,
        Status = "max_iterations"
    };
}

// Test the simple ReAct agent
var result = await SimpleReactAgent("What is RAG and what are its benefits?");

Console.WriteLine(new string('=', 80));
Console.WriteLine($"Final Answer: {result.Answer}");
Console.WriteLine($"Completed in {result.Iterations} iterations");
Console.WriteLine(new string('=', 80));

### 3.2 ReAct with Multi-Step Reasoning

In [ ]:
// Test with a question requiring multiple steps
var multiStepQuestion = "How many years after Chain-of-Thought was invented did the ReAct framework paper come out?";

Console.WriteLine("Multi-Step ReAct Example:");
Console.WriteLine(new string('=', 80));
var result2 = await SimpleReactAgent(multiStepQuestion);

Console.WriteLine(new string('=', 80));
Console.WriteLine($"Final Answer: {result2.Answer}");
Console.WriteLine($"Steps taken: {result2.Iterations}");
Console.WriteLine(new string('=', 80));

## 4. ReAct with Tool Usage <a id="tools"></a>

### 4.1 Enhanced ReAct with Better Tool Handling

In [ ]:
async Task<AgentResult> EnhancedReactAgent(string question, int maxIterations = 7, bool verbose = true)
{
    var systemPrompt = @"You are an AI assistant that uses the ReAct (Reasoning and Acting) framework.

For each step:
1. Think about what information you need
2. Choose and execute an appropriate action
3. Observe the result
4. Repeat until you can provide a final answer

Always use this exact format:
Thought: [your reasoning]
Action: tool_name(""argument"")

After receiving an observation, continue with another Thought/Action cycle or provide the final answer using finish().
";
    
    var userPrompt = $@"Available tools:
{GetToolDescriptions()}

Question: {question}

Let's solve this step by step using the ReAct framework.";
    
    var messages = new List<ChatRequestMessage>
    {
        new ChatRequestSystemMessage(systemPrompt),
        new ChatRequestUserMessage(userPrompt)
    };
    
    var trace = new List<TraceStep>();
    
    if (verbose)
    {
        Console.WriteLine(new string('=', 80));
        Console.WriteLine($"Question: {question}");
        Console.WriteLine(new string('=', 80));
        Console.WriteLine();
    }
    
    for (int iteration = 0; iteration < maxIterations; iteration++)
    {
        var requestOptions = new ChatCompletionsOptions
        {
            Temperature = 0.3f,
            MaxTokens = 400
        };
        
        foreach (var msg in messages)
        {
            requestOptions.Messages.Add(msg);
        }
        
        var response = await chat.CompleteChatAsync(model, requestOptions);
        var responseText = response.Value.Choices[0].Message.Content;
        messages.Add(new ChatRequestAssistantMessage(responseText));
        
        if (verbose)
        {
            Console.WriteLine($"Step {iteration + 1}:");
            Console.WriteLine(new string('-', 80));
            Console.WriteLine(responseText);
            Console.WriteLine();
        }
        
        var step = new TraceStep { Iteration = iteration + 1, Response = responseText };
        trace.Add(step);
        
        var actionInfo = ParseAction(responseText);
        
        if (actionInfo != null)
        {
            var observation = ExecuteAction(actionInfo.ToolName, actionInfo.Argument);
            
            step.Action = actionInfo;
            step.Observation = observation;
            
            if (verbose)
            {
                Console.WriteLine($"Observation: {observation}");
                Console.WriteLine(new string('=', 80));
                Console.WriteLine();
            }
            
            if (observation.Contains("FINAL_ANSWER:"))
            {
                var finalAnswer = observation.Replace("FINAL_ANSWER: ", "").Trim();
                return new AgentResult
                {
                    Answer = finalAnswer,
                    Iterations = iteration + 1,
                    Trace = trace,
                    Status = "success"
                };
            }
            
            messages.Add(new ChatRequestUserMessage($"Observation: {observation}"));
        }
        else
        {
            if (verbose)
            {
                Console.WriteLine("No action found, prompting for action...");
                Console.WriteLine();
            }
            messages.Add(new ChatRequestUserMessage("Please provide your next Thought and Action."));
        }
    }
    
    return new AgentResult
    {
        Answer = "Max iterations reached",
        Iterations = maxIterations,
        Trace = trace,
        Status = "max_iterations"
    };
}

// Test enhanced agent
var question3 = "What are the features of Azure AI Foundry and how many are there?";
var result3 = await EnhancedReactAgent(question3);

Console.WriteLine(new string('=', 80));
Console.WriteLine("FINAL RESULT");
Console.WriteLine(new string('=', 80));
Console.WriteLine($"Answer: {result3.Answer}");
Console.WriteLine($"Status: {result3.Status}");
Console.WriteLine($"Iterations: {result3.Iterations}");
Console.WriteLine(new string('=', 80));

## 5. Multi-Step ReAct Reasoning <a id="multi-step"></a>

### 5.1 Complex Multi-Step Problem

In [ ]:
var complexQuestion = @"If Chain-of-Thought was invented in 2022 and GPT-4 was released in 2023,
how many years passed between them? Also, what are GPT-4's capabilities?";

Console.WriteLine("Complex Multi-Step ReAct Example:");
var result4 = await EnhancedReactAgent(complexQuestion, maxIterations: 10);

Console.WriteLine(new string('=', 80));
Console.WriteLine("SUMMARY");
Console.WriteLine(new string('=', 80));
Console.WriteLine($"Question: {complexQuestion}");
Console.WriteLine($"\nAnswer: {result4.Answer}");
Console.WriteLine($"\nSteps taken: {result4.Iterations}");
Console.WriteLine($"Status: {result4.Status}");

Console.WriteLine("\n" + new string('=', 80));
Console.WriteLine("REASONING TRACE");
Console.WriteLine(new string('=', 80));
foreach (var step in result4.Trace)
{
    Console.WriteLine($"\nStep {step.Iteration}:");
    if (step.Action != null)
    {
        Console.WriteLine($"  Action: {step.Action.ToolName}({step.Action.Argument})");
        var obs = step.Observation.Length > 100 
            ? step.Observation.Substring(0, 100) + "..." 
            : step.Observation;
        Console.WriteLine($"  Result: {obs}");
    }
}

## 6. Error Handling and Recovery <a id="error-handling"></a>

### 6.1 ReAct with Error Recovery

In [ ]:
async Task<AgentResult> ReactWithRecovery(string question, int maxIterations = 8, bool verbose = true)
{
    var systemPrompt = @"You are an AI assistant using the ReAct framework.

When you encounter an error:
1. Acknowledge what went wrong
2. Adjust your approach
3. Try a different action or argument

Use this format:
Thought: [reasoning]
Action: tool_name(""argument"")

If you get an error, think about why and try again with a corrected approach.";
    
    var userPrompt = $@"Available tools:
{GetToolDescriptions()}

Question: {question}

Solve this using ReAct. If you encounter errors, learn from them and adjust.";
    
    var messages = new List<ChatRequestMessage>
    {
        new ChatRequestSystemMessage(systemPrompt),
        new ChatRequestUserMessage(userPrompt)
    };
    
    var trace = new List<TraceStep>();
    int errorsEncountered = 0;
    
    if (verbose)
    {
        Console.WriteLine(new string('=', 80));
        Console.WriteLine($"Question: {question}");
        Console.WriteLine(new string('=', 80));
        Console.WriteLine();
    }
    
    for (int iteration = 0; iteration < maxIterations; iteration++)
    {
        var requestOptions = new ChatCompletionsOptions
        {
            Temperature = 0.4f,
            MaxTokens = 400
        };
        
        foreach (var msg in messages)
        {
            requestOptions.Messages.Add(msg);
        }
        
        var response = await chat.CompleteChatAsync(model, requestOptions);
        var responseText = response.Value.Choices[0].Message.Content;
        messages.Add(new ChatRequestAssistantMessage(responseText));
        
        if (verbose)
        {
            Console.WriteLine($"Step {iteration + 1}:");
            Console.WriteLine(responseText);
            Console.WriteLine();
        }
        
        var step = new TraceStep { Iteration = iteration + 1, Response = responseText };
        trace.Add(step);
        
        var actionInfo = ParseAction(responseText);
        
        if (actionInfo != null)
        {
            var observation = ExecuteAction(actionInfo.ToolName, actionInfo.Argument);
            
            if (observation.Contains("Error"))
            {
                errorsEncountered++;
                if (verbose)
                {
                    Console.WriteLine($"❌ Error encountered: {observation}");
                    Console.WriteLine();
                }
            }
            
            step.Action = actionInfo;
            step.Observation = observation;
            step.IsError = observation.Contains("Error");
            
            if (verbose && !observation.Contains("Error"))
            {
                Console.WriteLine($"✓ Observation: {observation}");
                Console.WriteLine(new string('-', 80));
                Console.WriteLine();
            }
            
            if (observation.Contains("FINAL_ANSWER:"))
            {
                var finalAnswer = observation.Replace("FINAL_ANSWER: ", "").Trim();
                return new AgentResult
                {
                    Answer = finalAnswer,
                    Iterations = iteration + 1,
                    ErrorsEncountered = errorsEncountered,
                    Trace = trace,
                    Status = "success"
                };
            }
            
            messages.Add(new ChatRequestUserMessage($"Observation: {observation}"));
        }
    }
    
    return new AgentResult
    {
        Answer = "Max iterations reached",
        Iterations = maxIterations,
        ErrorsEncountered = errorsEncountered,
        Trace = trace,
        Status = "max_iterations"
    };
}

// Test with a question that might cause errors
var errorQuestion = "What is 2024 minus the release year of Azure AI Foundry?";

var result5 = await ReactWithRecovery(errorQuestion);

Console.WriteLine(new string('=', 80));
Console.WriteLine("RESULT");
Console.WriteLine(new string('=', 80));
Console.WriteLine($"Answer: {result5.Answer}");
Console.WriteLine($"Errors encountered and recovered from: {result5.ErrorsEncountered}");
Console.WriteLine($"Total iterations: {result5.Iterations}");
Console.WriteLine(new string('=', 80));

## 7. Advanced ReAct Patterns <a id="advanced"></a>

### 7.1 ReAct with Self-Reflection

In [ ]:
async Task<AgentResult> ReactWithReflection(string question, int maxIterations = 10, bool verbose = true)
{
    var systemPrompt = @"You are an advanced AI assistant using the ReAct framework with self-reflection.

Every few steps, pause to reflect:
- Am I making progress toward the answer?
- Is my current approach working?
- Should I try a different strategy?

Use these action types:
- Regular actions: tool_name(""argument"")
- Reflection: After every 2-3 actions, add a ""Reflection:"" statement

Format:
Thought: [reasoning]
Action: tool_name(""argument"")
[After observation]
Reflection: [evaluate progress and plan next steps]";
    
    var userPrompt = $@"Available tools:
{GetToolDescriptions()}

Question: {question}

Solve this using ReAct with periodic self-reflection.";
    
    var messages = new List<ChatRequestMessage>
    {
        new ChatRequestSystemMessage(systemPrompt),
        new ChatRequestUserMessage(userPrompt)
    };
    
    var trace = new List<TraceStep>();
    int actionsTaken = 0;
    
    if (verbose)
    {
        Console.WriteLine(new string('=', 80));
        Console.WriteLine($"Question: {question}");
        Console.WriteLine(new string('=', 80));
        Console.WriteLine();
    }
    
    for (int iteration = 0; iteration < maxIterations; iteration++)
    {
        var requestOptions = new ChatCompletionsOptions
        {
            Temperature = 0.4f,
            MaxTokens = 500
        };
        
        foreach (var msg in messages)
        {
            requestOptions.Messages.Add(msg);
        }
        
        var response = await chat.CompleteChatAsync(model, requestOptions);
        var responseText = response.Value.Choices[0].Message.Content;
        messages.Add(new ChatRequestAssistantMessage(responseText));
        
        if (verbose)
        {
            Console.WriteLine($"Step {iteration + 1}:");
            Console.WriteLine(new string('-', 80));
            Console.WriteLine(responseText);
            Console.WriteLine();
        }
        
        var step = new TraceStep 
        { 
            Iteration = iteration + 1, 
            Response = responseText
        };
        trace.Add(step);
        
        var actionInfo = ParseAction(responseText);
        
        if (actionInfo != null)
        {
            actionsTaken++;
            var observation = ExecuteAction(actionInfo.ToolName, actionInfo.Argument);
            
            step.Action = actionInfo;
            step.Observation = observation;
            
            if (verbose)
            {
                Console.WriteLine($"Observation: {observation}");
                Console.WriteLine(new string('=', 80));
                Console.WriteLine();
            }
            
            if (observation.Contains("FINAL_ANSWER:"))
            {
                var finalAnswer = observation.Replace("FINAL_ANSWER: ", "").Trim();
                int reflections = trace.Count(t => t.Response.ToLower().Contains("reflection:"));
                return new AgentResult
                {
                    Answer = finalAnswer,
                    Iterations = iteration + 1,
                    ActionsTaken = actionsTaken,
                    Reflections = reflections,
                    Trace = trace,
                    Status = "success"
                };
            }
            
            messages.Add(new ChatRequestUserMessage($"Observation: {observation}"));
            
            if (actionsTaken % 2 == 0 && actionsTaken > 0)
            {
                messages.Add(new ChatRequestUserMessage("Please reflect on your progress before continuing."));
            }
        }
    }
    
    return new AgentResult
    {
        Answer = "Max iterations reached",
        Iterations = maxIterations,
        ActionsTaken = actionsTaken,
        Trace = trace,
        Status = "max_iterations"
    };
}

// Test ReAct with reflection
var reflectionQuestion = @"What is the key innovation of the ReAct framework, and how does it relate to 
the components of RAG systems?";

var result6 = await ReactWithReflection(reflectionQuestion);

Console.WriteLine(new string('=', 80));
Console.WriteLine("SUMMARY");
Console.WriteLine(new string('=', 80));
Console.WriteLine($"Answer: {result6.Answer}");
Console.WriteLine($"Actions taken: {result6.ActionsTaken}");
Console.WriteLine($"Reflections: {result6.Reflections}");
Console.WriteLine($"Total iterations: {result6.Iterations}");
Console.WriteLine(new string('=', 80));

## 8. Summary and Best Practices <a id="summary"></a>

### Key Takeaways

**ReAct Framework Essentials:**

1. **Core Loop**: Thought → Action → Observation → (repeat)
2. **Interleaving**: Reasoning and acting happen together, not separately
3. **Transparency**: Reasoning traces explain every decision
4. **Flexibility**: Easy to add new tools and capabilities
5. **Robustness**: Can recover from errors through reasoning

### Implementation Best Practices

**Prompt Design:**
```
✓ Clear format specification (Thought/Action/Observation)
✓ Explicit tool descriptions
✓ System message defining the ReAct pattern
✓ Examples for complex scenarios
✗ Vague instructions
✗ Mixing action formats
```

**Action Parsing:**
- Use regex for consistent action extraction
- Handle variations in formatting gracefully
- Provide clear error messages for invalid actions
- Support both quoted and unquoted arguments

**Tool Design:**
- Keep tools focused and single-purpose
- Provide clear, informative descriptions
- Return structured, parseable results
- Include error handling in every tool
- Document expected input formats

**Error Handling:**
- Let the agent see and reason about errors
- Provide actionable error messages
- Allow the agent to retry with different approaches
- Set iteration limits to prevent infinite loops

**Reflection and Self-Correction:**
- Prompt for periodic reflection on progress
- Enable the agent to critique its own actions
- Allow strategy changes based on observations
- Track and learn from repeated errors

### Comparison with Other Patterns

| Pattern | Strengths | Weaknesses | Best For |
|---------|-----------|------------|----------|
| **ReAct** | Transparent, flexible, robust | More API calls, complex parsing | Multi-step tasks with tools |
| Chain-of-Thought | Simple, effective | No actions, less interactive | Pure reasoning tasks |
| Function Calling | Native support, reliable | Less transparent reasoning | Single-tool interactions |
| Agentic RAG | Knowledge-grounded | Requires good retrieval | Information-intensive tasks |

### Production Considerations

1. **Latency**: Each iteration adds API latency
   - Solution: Parallel tool execution when possible
   - Cache frequently used tool results

2. **Cost**: Multiple iterations increase token usage
   - Solution: Set reasonable iteration limits
   - Use smaller models for simple steps

3. **Reliability**: Parsing can fail on unexpected formats
   - Solution: Robust parsing with fallbacks
   - Retry with clearer instructions

4. **Monitoring**: Track agent behavior and success rates
   - Log all reasoning traces
   - Monitor tool usage patterns
   - Measure task completion rates

5. **Safety**: Agents can take unexpected actions
   - Solution: Validate tool inputs
   - Implement tool whitelisting
   - Add human approval for sensitive actions

### Advanced Techniques

1. **Multi-Agent ReAct**: Multiple ReAct agents collaborating
2. **Hierarchical ReAct**: Decompose into sub-tasks, each with ReAct
3. **ReAct + RAG**: Combine with retrieval for knowledge-grounded actions
4. **ReAct + Planning**: Add explicit planning phase before acting
5. **Meta-ReAct**: Agent reasons about which reasoning strategy to use

### When to Use ReAct

**Good Use Cases:**
- Tasks requiring multiple tool calls
- Problems needing exploratory approaches
- Scenarios where transparency is important
- Complex workflows with decision points
- Error-prone environments needing recovery

**Maybe Not Ideal:**
- Simple single-step questions
- Ultra-low latency requirements
- Pure generation tasks without tools
- Highly cost-sensitive applications

### Integration with Azure AI Foundry

ReAct works well with:
- **Azure AI Search**: As a retrieval tool
- **Function Calling**: For structured tool usage
- **Agent Framework**: As the reasoning engine
- **Evaluation Tools**: For measuring agent performance

### Next Steps

- Implement ReAct with real-world tools (APIs, databases, etc.)
- Combine ReAct with other techniques (RAG, planning)
- Build evaluation harnesses for agent quality
- Explore Microsoft Agent Framework integration
- Create domain-specific ReAct agents

### Additional Resources

- [ReAct Paper](https://arxiv.org/abs/2210.03629) - Original research
- [Microsoft Agent Framework](https://learn.microsoft.com/azure/ai-studio/)
- [LangChain ReAct](https://python.langchain.com/docs/modules/agents/agent_types/react)
- [Azure AI Foundry Agents](https://learn.microsoft.com/azure/ai-studio/concepts/agents)

## Practice Exercises

Try building these enhancements:

1. **Add New Tools**: Implement additional tools (web search, database query, etc.)
2. **Tool Composition**: Enable using output from one tool as input to another
3. **Parallel Actions**: Execute multiple independent actions simultaneously
4. **Memory**: Add persistent memory across ReAct sessions
5. **Planning**: Add an explicit planning phase before ReAct execution
6. **Multi-Agent**: Create multiple ReAct agents that collaborate

In [ ]:
// Your practice code here
// Try implementing one of the exercises above!